## Expected Goals (xG) Model - Data Import Pipeline

This notebook serves as the foundational data import pipeline for building an Expected Goals (xG) model using StatsBomb's open football data.<br/>
The xG model will predict the probability of a shot resulting in a goal based on various contextual factors such as shot distance, body part used, assist type, and defensive pressure.

We use the `statsbombpy` library to interact with StatsBomb's open-data GitHub repository. It abstracts the underlying JSON structure and URL handling, allowing straightforward access to competitions, matches, events, and lineups through simple function calls.


In [1]:
# Core data manipulation
import pandas as pd
from statsbombpy import sb

In [2]:
# file holding all the constants used in this project
from src.constants import *

# module made to get data
import src.get_data as gd

In [3]:
# Suppress StatsBomb authentication warnings for cleaner output
from statsbombpy.api_client import NoAuthWarning
import warnings
warnings.filterwarnings("ignore", category=NoAuthWarning)

In [4]:
# Enhanced pandas display for better data inspection
pd.set_option('display.max_columns', None)

### 1. Competitions Data

Competition data serves as the entry point to StatsBomb's hierarchical data structure. Each competition is uniquely identified by `competition_id` and `season_id`, which are required to access match-level data.

In [5]:
# Load and Filter Competitions
competitions = sb.competitions()

# Filter: keep only male competitions
competitions = competitions[competitions['competition_gender']=='male'].sort_values('season_name', ascending=False)

# Filter: keep seasons from 2014 onward
competitions = competitions[competitions['season_name']>'2014']

# Filter: exclude Indian Super League
competitions = competitions[competitions['competition_name']!='Indian Super league']

# Filter: exclude Major League Soccer
competitions = competitions[competitions['competition_name']!='Major League Soccer']

For our analysis we need only the basic info about the each competition, so various columns will be dropped in order to keep the needed ones only.
<br>I have decided to use data from the 2014 and up to allow some recency and the availability of data for that season for the top 5 leagues.

In [6]:
# Drop unnecessary columns
competitions.drop(COMPETITIONS_COLUMNS_DROP,
                  axis=1,
                  inplace=True)

# Reset index after filtering
competitions.reset_index(drop=True,
                         inplace=True)

In [7]:
competitions

,competition_id,season_id,country_name,competition_name,competition_international,season_name
0,223,282,South America,Copa America,True,2024
1,55,282,Europe,UEFA Euro,True,2024
2,9,281,Germany,1. Bundesliga,False,2023/2024
3,1267,107,Africa,African Cup of Nations,True,2023
4,7,235,France,Ligue 1,False,2022/2023
5,43,106,International,FIFA World Cup,True,2022
6,7,108,France,Ligue 1,False,2021/2022
7,11,90,Spain,La Liga,False,2020/2021
8,55,43,Europe,UEFA Euro,True,2020
9,11,42,Spain,La Liga,False,2019/2020


The filtering process yields 25 competition-seasons across major footballing regions:

- <b>European Leagues</b>: Premier League, La Liga, Serie A, Bundesliga, Ligue 1
- <b>International Tournaments</b>: FIFA World Cup, UEFA Euro, Copa America
- <b>Continental Competitions</b>: UEFA Champions League, African Cup of Nations

In [ ]:
competitions.to_csv('data/raw/competitions.csv', index=False)

### 2. Matches
Match data bridges competitions and events, providing:

- <b>Match Context</b>: Date, teams, venue, officials
- <b>Result Information</b>: Scores and competition stage
- <b>Access Keys</b>: `match_id` for event-level data retrieval

In [9]:
df_matches = gd.get_matches(competitions)
df_matches.head()

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version
0,3943077,2024-07-15,04:15:00.000,South America - Copa America,2024,Argentina,Colombia,1,0,6,Final,Hard Rock Stadium,Raphael Claus,Lionel Sebastián Scaloni,Néstor Gabriel Lorenzo,1.1.0
1,3943076,2024-07-14,03:00:00.000,South America - Copa America,2024,Canada,Uruguay,2,2,6,3rd Place Final,Bank of America Stadium,Alexis Herrera,Jesse Marsch,Marcelo Alberto Bielsa Caldera,1.1.0
2,3942852,2024-07-11,03:00:00.000,South America - Copa America,2024,Uruguay,Colombia,0,1,5,Semi-finals,Bank of America Stadium,César Arturo Ramos Palazuelos,Marcelo Alberto Bielsa Caldera,Néstor Gabriel Lorenzo,1.1.0
3,3942785,2024-07-10,03:00:00.000,South America - Copa America,2024,Argentina,Canada,2,0,5,Semi-finals,MetLife Stadium,Piero Maza Gomez,Lionel Sebastián Scaloni,Jesse Marsch,1.1.0
4,3942416,2024-07-07,01:00:00.000,South America - Copa America,2024,Colombia,Panama,5,0,4,Quarter-finals,State Farm Stadium,Maurizio Mariani,Néstor Gabriel Lorenzo,Thomas Christiansen Tarín,1.1.0


In [10]:
df_matches.groupby('competition').size()

competition
Africa - African Cup of Nations     52
England - Premier League           380
Europe - Champions League            5
Europe - UEFA Euro                 102
France - Ligue 1                   435
Germany - 1. Bundesliga            340
International - FIFA World Cup     128
Italy - Serie A                    380
South America - Copa America        32
Spain - La Liga                    590
dtype: int64

We end up with a total of 2444 matches
- 5 UCL finals
- 380 for Serie A and Premier League (2014-2015 Free Data)
- 340 for Bundesliga (2014-2015 Free Data + Bayer Leverkusen Invincible free data) 
- 435 and 590 for Ligue 1 and La Liga because of Messi's career open data
- In addition to Other international competitions

In [ ]:
df_matches.to_csv('data/raw/matches.csv', index=False)

### 3. Lineups

Lineup data provides player-specific information including:

- <b>Positional Data</b>: Starting positions and in-game movements
- <b>Disciplinary Records</b>: Cards received during matches
- <b>Squad Information</b>: Jersey numbers and nationalities

In [11]:
df_lineups = gd.get_lineups(df_matches)

[0] Processed match_id 3943077
[50] Processed match_id 3930179
[100] Processed match_id 3895320
[150] Processed match_id 3920412
[200] Processed match_id 3837752
[250] Processed match_id 3857292
[300] Processed match_id 3773631
[350] Processed match_id 3794692
[400] Processed match_id 303696
[450] Processed match_id 22912
[500] Processed match_id 7572
[550] Processed match_id 9754
[600] Processed match_id 3754053
[650] Processed match_id 3754093
[700] Processed match_id 3754223
[750] Processed match_id 3754343
[800] Processed match_id 3754016
[850] Processed match_id 3754139
[900] Processed match_id 3754239
[950] Processed match_id 3754333
[1000] Processed match_id 3890545
[1050] Processed match_id 3890492
[1100] Processed match_id 3890442
[1150] Processed match_id 3890387
[1200] Processed match_id 3890335
[1250] Processed match_id 3890284
[1300] Processed match_id 3825839
[1350] Processed match_id 3825803
[1400] Processed match_id 3825711
[1450] Processed match_id 3825623
[1500] Proce

In [12]:
df_lineups.head()

,player_id,player_name,player_nickname,jersey_number,country,cards,positions,match_id
0,2995,Ángel Fabián Di María Hernández,Ángel Di María,11,Argentina,[],"[{'position_id': 12, 'position': 'Right Midfie...",3943077
1,3090,Nicolás Hernán Otamendi,Nicolás Otamendi,19,Argentina,[],"[{'position_id': 12, 'position': 'Right Midfie...",3943077
2,3313,Giovani Lo Celso,None,16,Argentina,"[{'time': '11:26', 'card_type': 'Yellow Card',...","[{'position_id': 11, 'position': 'Left Defensi...",3943077
3,5503,Lionel Andrés Messi Cuccittini,Lionel Messi,10,Argentina,[],"[{'position_id': 22, 'position': 'Right Center...",3943077
4,5507,Nicolás Alejandro Tagliafico,Nicolás Tagliafico,3,Argentina,[],"[{'position_id': 6, 'position': 'Left Back', '...",3943077


In [ ]:
df_lineups.to_csv('data/raw/lineups.csv', index=False)

<b><i><u>Note</b></u>: While comprehensive, lineup data is not directly utilized in the xG model but serves as supplementary information for player-level analysis and potential future model enhancements.</i>

### 4. Events

Event data forms the foundation of our xG model, containing:

- <b>Shot Events</b>: Location, body part, technique, outcome
- <b>Contextual Information</b>: Assist type, defensive pressure, game state
- <b>Temporal Data</b>: Match minute, sequence timing
- <b>Spatial Data</b>: Coordinates, angles, distances

Due to the massive size of event data (~2.4M+ rows), we implement:

- **Batch Processing**: Load data in chunks of 500 matches to avoid memory overflow during iteration.
- **Selective Column Import**: Use a predefined list of relevant columns (`EVENT_COLUMNS_SELECT`) to ensure schema consistency across matches (StatsBomb's JSON structure may vary slightly: 120–122 columns).
- **Custom CSV Separator**: Use `;` as the CSV delimiter to prevent parsing issues when reading with PySpark, especially for columns like `shot_freeze_frame` which store JSON-like Python dicts containing commas. Without this, default comma-separated CSV parsing breaks nested structures.

In [ ]:
# Process events in manageable batches
start_i = 0
end_i = 500 # Adjust based on memory constraints

df_events = gd.get_events(df_matches, EVENT_COLUMNS_SELECT, start_i, end_i)

[0] Processed match_id=3943077 (4108 rows)
Completed 10 matches.
Total rows: 31102
Columns: 122
Null match_id values: 0


The ```type``` holds the type of event the row represents

In [26]:
df_events[df_events['type']=='Shot'].shape[0]

274

For a typical batch, expect:

- **Shot Events**: ~15,000-20,000 shot records
- **Total Events**: ~500,000+ all event types
- **Key Columns**: 122 columns including shot-specific attributes

In [ ]:
# Export with custom separator to handle nested data
df_events.to_csv(f'data/raw/events_{start_i}_{end_i}.csv', index=False, sep=';')

### 5. Frames

Frame data represents the cutting edge of football analytics, providing:

- **Player Tracking**: X,Y coordinates for all players
- **Defensive Pressure**: Proximity of defenders to shooter
- **Space Analysis**: Available space and passing lanes
- **Tactical Context**: Team shape and player positioning

Availability Constraint: Frame data is available for only a subset of competitions in StatsBomb's free dataset, limiting its immediate utility for our comprehensive xG model.

In [ ]:
# Extract available frame data
df_frames = gd.get_frames(df_matches)

In [ ]:
df_frames.to_csv('data/raw/frames.csv', index=False)

**Note**: For a detailed explanation of the StatsBomb data structure and event types, refer to the documentation in <a href='docs/'>docs</a>. This includes official StatsBomb PDFs and schema references.